In [8]:
#imports necesarios
import requests
import time
from pymongo import MongoClient

In [9]:
#conexion a mongodb
MONGO_URI = "mongodb://172.17.0.2:27017/" 
#Configuracoin da base de datos
DB_NAME = "citybikes_db"
COLLECTION_NAME = "stations"

In [10]:
#conexion coa api
API_URL = "https://api.citybik.es/v2/networks"

INTERVAL = 300 #5 minutos

In [11]:
def conectar_mongodb(): #conexion coa bd
    try:
        cliente = MongoClient(MONGO_URI)
        db = cliente[DB_NAME]
        print("Conexión con MongoDB feita")
        return db[COLLECTION_NAME]
    except Exception as e:
        print(f"Problema ao conectar con MongoDB: {e}")
        raise

def obter_datos_api(): #obtemos  os datos da api
    try:
        resposta = requests.get(API_URL)
        resposta.raise_for_status()
        return resposta.json()
    except requests.exceptions.RequestException as e:
        print(f"Problema ao obter datos da API: {e}")
        return None
    
def almacenar_datos_mongodb(coleccion, datos): #almaceamos os datos obtidos
    if datos:
        datos['timestamp'] = time.time() #para ver cando se gardaron os datos
        coleccion.insert_one(datos)
        print("Datos almacenados en Mongo.")
    else:
        print("Non hai datos.")


def executar(): #fai o bucle para a obtencion dos datos e o seu almaceamento
    coleccion = conectar_mongodb()

    try:
        while True:
            datos = obter_datos_api()
            almacenar_datos_mongodb(coleccion, datos)
            time.sleep(INTERVAL)
    except KeyboardInterrupt:
        print("Execución parada polo usuario.")
    finally:
        print("Fin.")

#para evitar execucions non desexadas:
if __name__ == "__main__":
    executar()


Conexión con MongoDB feita
Datos almacenados en Mongo.
Execución parada polo usuario.
Fin.
